<a href="https://colab.research.google.com/github/rugggg/mastering_llms_notes/blob/main/axolotl_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
# Check so there is a gpu available, a T4(free tier) is enough to run this notebook
assert (torch.cuda.is_available()==True)

In [2]:
!pip install torch=="2.1.2"
!pip install -e git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
!pip install flash-attn=="2.5.0"
!pip install deepspeed=="0.13.1"!pip install mlflow=="2.13.0"

Obtaining axolotl from git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
  Updating ./src/axolotl clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 05b0bd08d229ee28cd3f11098d5b178f2ce441b6
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/lm-sys/FastChat.git (to revision 27a05b04a35510afb1d767ae7e5990cbd278f8fe) to /tmp/pip-install-ax4uykw7/fschat_7b25e7ee093b42f39f12b2a9843572c0
  Running command git clone --filter=blob:none --quiet https://github.com/lm-sys/FastChat.git /tmp/pip-install-ax4uykw7/fschat_7b25e7ee093b42f39f12b2a9843572c0
  Running command git rev-parse -q --verify 'sha^27a05b04a35510afb1d767ae7e5990cbd278f8fe'
  Running command git fetch -q https://github.com/lm-sys/FastChat.git 27a05b04a35510afb1d767ae7e5990cbd278f8fe
  Running command git checkout -q 27a05b04a35510afb1d767ae7e5990cbd278f8fe
  Resolved https://github.com/lm-sys/FastChat.git to commit 27a05b04a35510afb1d767ae7e5990cbd278f8fe
  Install

In [6]:
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.14.2-py3-none-any.whl size=1432232 sha256=32b28d2e7cfe273442b00eae0383e772417d48e0c1bfab6c4f01a96b30889e98
  Stored in directory: /root/.cache/pip/wheels/ea/7c/43/bed44d8414c099ff962b754f425f7ff77cc623cc8a98e0da70
Successfully built deepspeed


In [7]:
import yaml

# Your YAML string
yaml_string = """
base_model: TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T
model_type: LlamaForCausalLM
tokenizer_type: LlamaTokenizer

load_in_8bit: false
load_in_4bit: true
strict: false

datasets:
  - path: mhenrichsen/alpaca_2k_test
    type: alpaca
dataset_prepared_path:
val_set_size: 0.05
output_dir: ./outputs/qlora-out

adapter: qlora
lora_model_dir:

sequence_len: 4096
sample_packing: true
eval_sample_packing: false
pad_to_sequence_len: true

lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 4
micro_batch_size: 2
num_epochs: 4
optimizer: paged_adamw_32bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

warmup_steps: 10
evals_per_epoch: 4
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:

"""

# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
file_path = 'test_axolotl.yaml'

# Write the YAML file
with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

In [4]:

# Buy using the ! the comand will be executed as a bash command
!accelerate launch -m axolotl.cli.train /content/test_axolotl.yaml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-06-01 04:26:42,592] [INFO] [datasets.<module>:58] [PID:2314] PyTorch version 2.1.2 available.
[2024-06-01 04:26:42,593] [INFO] [datasets.<module>:70] [PID:2314] Polars version 0.20.2 available.
[2024-06-01 04:26:42,594] [INFO] [datasets.<module>:105] [PID:2314] TensorFlow version 2.15.0 available.
[2024-06-01 04:26:42,595] [INFO] [datasets.<module>:118] [PID:2314] JAX version 0.4.26 available.
2024-06-01 04:26:46.255482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
202

In [5]:

# Buy using the ! the comand will be executed as a bash command
!accelerate launch -m axolotl.cli.inference /content/test_axolotl.yaml \
    --qlora_model_dir="./qlora-out" --gradio

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-06-01 04:28:14,313] [INFO] [datasets.<module>:58] [PID:2801] PyTorch version 2.1.2 available.
[2024-06-01 04:28:14,314] [INFO] [datasets.<module>:70] [PID:2801] Polars version 0.20.2 available.
[2024-06-01 04:28:14,315] [INFO] [datasets.<module>:105] [PID:2801] TensorFlow version 2.15.0 available.
[2024-06-01 04:28:14,316] [INFO] [datasets.<module>:118] [PID:2801] JAX version 0.4.26 available.
2024-06-01 04:28:15.952319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
202